In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
import json
import pandas as pd

In [2]:
#한글패치
import platform
from matplotlib import font_manager , rc

if platform.system() == 'Darwin':
  rc('font' , family = 'AppleGothic')
elif platform.system() == 'Windows':
  path = 'C:/Windows/Fonts/malgun.ttf'
  font_name = font_manager.FontProperties(fname = path).get_name()
  rc('font' , family = font_name)
else:
  print('모름')

In [3]:

#필요한 파일 로드
seoul_gdf = gpd.read_file('C:/k_digital/Pythonbasic/LARD_ADM_SECT_SGG_서울/LARD_ADM_SECT_SGG_11.shp', crs='EPSG:4326')
seoul_gdf1 = gpd.read_file('C:/Users/NT551_11TH/Desktop/4652_조민우_반출데이터_20231011/B402. 서울시 장애인시설 공간데이터/B402. 서울시 장애인시설 공간데이터/2. 파일데이터/2019/TL_DSPSN_FCLTY_ST_2019.shp', crs='EPSG:4326')
geo_path = '02. skorea_municipalities_geo_simple.json'
geo_str = json.load(open(geo_path , encoding = 'UTF-8'))
total = pd.read_csv('장애인 비율 통계.csv' , encoding = 'euc-kr')
a = pd.read_excel('위도경도.xlsx')

In [5]:
#a 테이블에서 위도와 경도 , 그리고 구 이름만 가져오기
a = a.iloc[:,[2,3,4]]

#total과 테이블을 합치기 위해 기준이 되는 열 이름을 통일
a.rename(columns = {'city' : 'GU_NAM'} , inplace = True)

In [6]:
#GU_NAM이라는 열 이름을 기준으로 total과 a를 통합.
total = pd.merge(total , a , on = 'GU_NAM')

In [9]:
total

,GU_NAM,총인구,장애인인구,장애인비율,longitude,latitude
0,종로구,161869,18201,11.244278,126.981642,37.600000
1,중구,136488,17136,12.554950,126.999642,37.561003
2,용산구,245185,24075,9.819116,126.967522,37.536094
3,성동구,308979,35058,11.346402,127.039000,37.560611
4,광진구,366972,37689,10.270266,127.084533,37.535739
5,동대문구,363023,47613,13.115698,127.070000,37.580000
6,중랑구,402024,61083,15.193869,127.094778,37.603806
7,성북구,454744,53061,11.668323,127.020333,37.600000
8,강북구,317695,52551,16.541337,127.027719,37.636956
9,도봉구,335631,46350,13.809809,127.049522,37.665833


In [7]:
#지도를 그려본 결과 , 글씨가 많이 벗어나 있는 것들이 많았음. 세부적인 조정
total.loc[total['GU_NAM']=='양천구' , 'latitude'] = 37.52
total.loc[total['GU_NAM']=='서초구' , 'longitude'] = 127.00
total.loc[total['GU_NAM']=='동대문구' , 'latitude'] = 37.58
total.loc[total['GU_NAM']=='동대문구' , 'longitude'] = 127.07
total.loc[total['GU_NAM']=='노원구' , 'longitude'] = 127.07
total.loc[total['GU_NAM']=='은평구' , 'latitude'] = 37.61
total.loc[total['GU_NAM']=='종로구' , 'latitude'] = 37.60
total.loc[total['GU_NAM']=='성북구' , 'latitude'] = 37.60

In [8]:
#지도 그리기
seoul_map = folium.Map(location=[37.5665, 126.9780], zoom_start=10)
folium.Choropleth(
    geo_data=geo_str,
    data=total,
    columns=['GU_NAM', '장애인인구'],
    fill_color='BuGn',
    key_on='feature.properties.name', # json 파일과 total의 GU_NAM과 일치시킬 key값 찾기.
).add_to(seoul_map)

#지도의 구역마다 그 곳에 해당하는 구 이름을 text로 삽입.
for idx, row in total.iterrows():
    text = row['GU_NAM']
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        icon=folium.DivIcon(html=f'<div>{text}</div>') # 임의의 text값을 넣기 위해 html 형식으로 삽입.
    ).add_to(seoul_map)

folium.TileLayer('stamenwatercolor').add_to(seoul_map)
seoul_map

In [176]:
seoul_map.save('장애인 인구 구별 분포.html')

In [10]:
seoul_map = folium.Map(location=[37.5665, 126.9780], zoom_start=10)
folium.Choropleth(
    geo_data=geo_str,
    data=total,
    columns=['GU_NAM', '장애인비율'],
    fill_color='BuGn',
    key_on='feature.properties.name',
).add_to(seoul_map)

for idx, row in total.iterrows():
    text = row['GU_NAM']
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        icon=folium.DivIcon(html=f'<div>{text}</div>') 
    ).add_to(seoul_map)

folium.TileLayer('stamenwatercolor').add_to(seoul_map)
seoul_map

In [179]:
seoul_map.save('장애인 비율 구별 분포.html')

In [11]:
folium.GeoJson(seoul_gdf1).add_to(seoul_map) # seoul_gdf1은 장애인 편의시설의 위치데이터이다. 그것을 Marker형식으로 추가
seoul_map

In [182]:
seoul_map.save('마커 찍힌 구별.html')

import geopandas as gpd
import matplotlib.pyplot as plt


gdf = gpd.read_file('')


fig, ax = plt.subplots(1, 1)
gdf.plot(column='인구수', cmap='OrRd', linewidth=0.8, ax=ax, edgecolor='0.8', legend=True)


sm = plt.cm.ScalarMappable(cmap='OrRd', norm=plt.Normalize(vmin=gdf['인구수'].min(), vmax=gdf['인구수'].max()))
sm._A = []
cbar = fig.colorbar(sm)


plt.title("인구수에 따른 지도 시각화")
ax.set_axis_off()

결과 출력
plt.show()